<a href="https://colab.research.google.com/github/dylanJzch/Predict-Credit-Card-Default/blob/main/11predict_credit_card_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import numpy as np

# New Section

In [48]:
from google.colab import drive
drive.mount('/content/drive')
df_raw = pd.read_excel("/content/drive/MyDrive/BDML/default_of_credit_card_clients.xls")

data = df_raw.iloc[1:,1:] #remove the first row and first column
data.columns = df_raw.iloc[0].values[1:]
print(data.shape)
data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(30000, 24)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [36]:
## PARTITION by age
female_over_30 = data[(data['X2'] == 2) & (data['X5'] > 30)] ## Female over 30
female_under_30 = data[(data['X2'] == 2) & (data['X5'] < 30)] ## Female under 30

male_over_30 = data[(data['X2'] == 1) & (data['X5'] > 30)] ## Male over 30
male_under_30 = data[(data['X2'] == 1) & (data['X5'] < 30)] ## Male under 30
##-------------------------------------------------------------------------------------------##
## Partition by education
## Female over 30: 
female_over_30_grad = female_over_30[female_over_30['X3'] == 1]  # graduate school
female_over_30_uni = female_over_30[female_over_30['X3'] == 2]  # university
female_over_30_highschool = female_over_30[female_over_30['X3'] == 3]  # high school
female_over_30_others = female_over_30[female_over_30['X3'] == 4]  # others
## Female under 30: 
female_under_30_grad = female_under_30[female_under_30['X3'] == 1]  # graduate school
female_under_30_uni = female_under_30[female_under_30['X3'] == 2]  # university
female_under_30_highschool = female_under_30[female_under_30['X3'] == 3]  # high school
female_under_30_others = female_under_30[female_under_30['X3'] == 4]  # others
## Male over 30: 
male_over_30_grad = male_over_30[male_over_30['X3'] == 1]  # graduate school
male_over_30_uni = male_over_30[male_over_30['X3'] == 2]  # university
male_over_30_highschool = male_over_30[male_over_30['X3'] == 3]  # high school
male_over_30_others = male_over_30[male_over_30['X3'] == 4]  # others
## Male under 30: 
male_under_30_grad = male_under_30[male_under_30['X3'] == 1]  # graduate school
male_under_30_uni = male_under_30[male_under_30['X3'] == 2]  # university
male_under_30_highschool = male_under_30[male_under_30['X3'] == 3]  # high school
male_under_30_others = male_under_30[male_under_30['X3'] == 4]  # others 


In [40]:
# Preparing data for the feedforward network
X = data.drop(['Y', 'Unnamed: 0'], axis=1)
y = data['Y']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [41]:
# Train the feedforward network
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the feedforward network
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Feedforward Network Accuracy: {accuracy}")

Epoch 1/20
675/675 [==============================] - 2s 2ms/step - loss: 0.4168 - accuracy: 0.8247 - val_loss: 0.4534 - val_accuracy: 0.8058
Epoch 2/20
675/675 [==============================] - 1s 2ms/step - loss: 0.4162 - accuracy: 0.8265 - val_loss: 0.4605 - val_accuracy: 0.8046
Epoch 3/20
675/675 [==============================] - 1s 2ms/step - loss: 0.4152 - accuracy: 0.8257 - val_loss: 0.4554 - val_accuracy: 0.8079
Epoch 4/20
675/675 [==============================] - 2s 3ms/step - loss: 0.4144 - accuracy: 0.8239 - val_loss: 0.4571 - val_accuracy: 0.8087
Epoch 5/20
675/675 [==============================] - 2s 3ms/step - loss: 0.4140 - accuracy: 0.8251 - val_loss: 0.4557 - val_accuracy: 0.8046
Epoch 6/20
675/675 [==============================] - 2s 3ms/step - loss: 0.4136 - accuracy: 0.8253 - val_loss: 0.4578 - val_accuracy: 0.8058
Epoch 7/20
675/675 [==============================] - 1s 2ms/step - loss: 0.4130 - accuracy: 0.8266 - val_loss: 0.4579 - val_accuracy: 0.8071
Epoch 

In [42]:
# Preparing data for the LSTM
X_time_series = data[['X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']]
X_time_series = np.array(X_time_series).reshape(X_time_series.shape[0], 3, 6)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_time_series, y, test_size=0.2, random_state=42)


In [43]:
# Define the LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 6)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [44]:
# Train the LSTM
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the LSTM
loss, accuracy = model.evaluate(X_test, y_test)
print(f"LSTM Accuracy: {accuracy}")

Epoch 1/20
675/675 [==============================] - 4s 4ms/step - loss: 267.8267 - accuracy: 0.6787 - val_loss: 78.5086 - val_accuracy: 0.7029
Epoch 2/20
675/675 [==============================] - 2s 4ms/step - loss: 69.2337 - accuracy: 0.6933 - val_loss: 42.6580 - val_accuracy: 0.7258
Epoch 3/20
675/675 [==============================] - 3s 4ms/step - loss: 43.8928 - accuracy: 0.6985 - val_loss: 29.1950 - val_accuracy: 0.6525
Epoch 4/20
675/675 [==============================] - 4s 5ms/step - loss: 30.4328 - accuracy: 0.7045 - val_loss: 25.4166 - val_accuracy: 0.7492
Epoch 5/20
675/675 [==============================] - 2s 3ms/step - loss: 20.7830 - accuracy: 0.7078 - val_loss: 16.4070 - val_accuracy: 0.7375
Epoch 6/20
675/675 [==============================] - 2s 4ms/step - loss: 15.3206 - accuracy: 0.7006 - val_loss: 12.9301 - val_accuracy: 0.5767
Epoch 7/20
675/675 [==============================] - 3s 4ms/step - loss: 10.6475 - accuracy: 0.7032 - val_loss: 7.8599 - val_accuracy: